In [ ]:
!pip3 install requests

In [3]:
import requests 
import random
import string

base_url = "https://du-backend-t437.onrender.com/"


ir_ids = [
    "IM7216", "IM6599", "IM1579", "IM9252", "IM2004", "IM4230", "IM7990", "IM0262",
    "IM1781", "IM1332", "IM9002", "IM4546", "IM4927", "IM6392", "IM2779", "IM9215",
    "IM8947", "IM8361", "IM2499", "IM0424", "IM6678", "IM9375", "IM8657", "IM5852", "IM1530"
]

/Users/snehith/PycharmProjects/du_backend/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:

def create_ir(ir_id):
    """Send IR ID to the API."""
    path = "/api/add_ir_id"
    end_point = f"{base_url}{path}"
    payload = {"ir_id": ir_id}

    try:
        response = requests.post(end_point, json=payload, timeout=10)
        response.raise_for_status()
        print(f"Created {ir_id}: {response.json()}")
    except requests.exceptions.Timeout:
        print(f"{ir_id}: Request timed out.")
    except requests.exceptions.ConnectionError:
        print(f"{ir_id}: Failed to connect to the server.")
    except requests.exceptions.HTTPError as err:
        print(f"{ir_id}: HTTP error occurred: {err.response.text}")
    except Exception as e:
        print(f"{ir_id}: An error occurred: {e}")


for ir_id in ir_ids:
    create_ir(ir_id)


In [ ]:


# access_distribution = (
#     [1] +                  # Admin
#     [2] * 5 +               # LDC
#     [3] * 8 +               # LS
#     [4] * 4 +               # GC
#     [5] * (len(ir_ids) - (1 + 5 + 8 + 4))  # IR
# )

# def register_ir(ir_id, access_level):
#     path = "/api/register_new_ir"
#     end_point = f"{base_url}{path}"
#     data = {
#         "ir_id": ir_id,
#         "ir_name": f"Test User {ir_id}",
#         "ir_email": f"{ir_id.lower()}@example.com",
#         "ir_password": "TestPass123",
#         "ir_access_level": access_level
#     }
    
#     try:
#         response = requests.post(
#             end_point,
#             json=data,
#             timeout=10
#         )
        
#         if response.ok:
#             try:
#                 print(f"✅ Registered {ir_id} (Access: {access_level})")
#             except ValueError:
#                 print(f"✅ Registered {ir_id} (Access: {access_level}) ")
#         else:
#             try:
#                 print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} error: {response.json()}")
#             except ValueError:
#                 print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} error: {response.text}")
    
#     except requests.exceptions.RequestException as e:
#         print(f"🚫 Failed {ir_id}: {e}")

# def populate_test_irs():
#     for ir_id, access_level in zip(ir_ids, access_distribution):
#         register_ir(ir_id, access_level)

# if __name__ == "__main__":
#     populate_test_irs()

In [ ]:
access_distribution = (
    [1] +                  # Admin (1 user)
    [2] * 5 +              # LDC (5 users)
    [3] * 8 +              # LS (8 users)
    [4] * (len(ir_ids) - (1 + 5 + 8))  # GC (Remaining users)
)

def register_ir(ir_id, access_level):
    path = "/api/register_new_ir"
    end_point = f"{base_url}{path}"
    data = {
        "ir_id": ir_id,
        "ir_name": f"Test User {ir_id}",
        "ir_email": f"{ir_id.lower()}@example.com",
        "ir_password": "TestPass123",
        "ir_access_level": access_level
    }
    
    try:
        response = requests.post(
            end_point,
            json=data,
            timeout=10
        )
        
        if response.ok:
            print(f"✅ Registered {ir_id} (Access: {access_level})")
        else:
            try:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} | {response.json()}")
            except ValueError:
                print(f"❌ Error {ir_id} (Access: {access_level}) → {response.status_code} | {response.text}")
    
    except requests.exceptions.RequestException as e:
        print(f"🚫 Failed {ir_id}: {e}")

def populate_test_irs():
    for ir_id, access_level in zip(ir_ids, access_distribution):
        register_ir(ir_id, access_level)

if __name__ == "__main__":
    populate_test_irs()

In [ ]:
def getAllRegisteredIr():
    path = "/api/irs"
    end_point = f"{base_url}{path}"
    response = requests.get(end_point)
    
    if response.ok:
        try:
            print("Result:", response.json())
        except ValueError:
            print("Result (non-JSON):", response.text)
    else:
        try:
            print("Error:", response.json())
        except ValueError:
            print("Error (non-JSON):", response.text)

getAllRegisteredIr()
